# Concurrencia vs Paralelismo

Concurrency signfica que hay 2 o más procesos ejecutandose dentro del mismo periodo de tiempo. Paralelismo signfica que hay 2 o más procesos ejecutandose en el mismo momento. Paralelismo es un caso especial de concurrencia.


# Multiprocessing

Como ya vimos en la clase pasada, hilos en Python no está diseñado para funcionar en computadoras con más de un core, por ello GIL es necesario porque Python por sí solo no es "thread-safe". 

Multiprocessing nos permite crear programas que puedan correr concurrentemente (bypaseando GIL) y usar todos los core de la CPU. Multiprocessing es sintácticamente parecido a Threading pero implementados de manera diferente. Multiprocessing le da a cada proceso su propio interprete de Python cada uno con su propio GIL.


In [5]:
import multiprocessing
def spawn(i):
    print(i)

if __name__ == '__main__':
    for i in range(5):
        p = multiprocessing.Process(target=spawn, args=(i,))
        p.start()
        p.join()

0
1
2
3
4


Cuando usar multiprocessing:
* Cuando se usa mucho consumo de CPU.
* Cuando tiene una gran cantidad de uso de red o entrada/salida
* Si se está usando UI.